
# Transformer model for erroneous data detection

## Loading Data

In [1]:
#pip install tensorflow_hub
#pip install tensorflow==1.15

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tabReformer import run_classifier
from tabReformer import optimization
from tabReformer import tokenization

from tensorflow import keras
import os
import re

We'll set an output directory location to store our model output and checkpoints.

In [3]:
# Set the output directory for saving model file
#@markdown Whether or not to clear/delete the directory and create a new one

OUTPUT_DIR = 'err_detect_output'#@param {type:"string"}

DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
    try:
        tf.gfile.DeleteRecursively(OUTPUT_DIR)
    except:
        # Doesn't matter if the directory didn't exist
        pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: err_detect_output *****


## Data Augmentation

In [4]:
def create_dirty_data(dataset_name):
    clean_df = pd.read_csv(Config.get("datapool.folder") + '/'+dataset_name+'/clean.csv', header=0, dtype=object, na_filter=False)
    print(clean_df.shape)
    dfupdate=clean_df#.sample(3000)

    for i, row in dfupdate.iterrows():
        if i%10==0 or i%9==0:
            column_index = random.randint(0,len(dfupdate.columns)-1)
            #print (column_index)
            x = random.randint(0,3)
            #print(x)
            if x==0: #shuffle
                dfupdate.at[i,clean_df.columns[column_index]] = ''.join(random.sample(str(dfupdate.at[i,clean_df.columns[column_index]]),len(str(dfupdate.at[i,clean_df.columns[column_index]]))))
                #print(dfupdate.at[i,clean_df.columns[column_index]])
            elif x==1: #add a random character at the end
                dfupdate.at[i,clean_df.columns[column_index]] = str(dfupdate.at[i,clean_df.columns[column_index]]) + random.choice(string.ascii_letters)
                #print(dfupdate.at[i,clean_df.columns[column_index]])
            elif x==2: #remove value
                dfupdate.at[i,clean_df.columns[column_index]] = ''
                #print(dfupdate.at[i,clean_df.columns[column_index]])
            else: #add a random character in the start
                dfupdate.at[i,clean_df.columns[column_index]] = random.choice(string.ascii_letters) + str(dfupdate.at[i,clean_df.columns[column_index]])
                #print(dfupdate.at[i,clean_df.columns[column_index]])

    print(dfupdate.shape)
    file_name=dataset_name+'_dirty.csv'
    dfupdate.to_csv(file_name, index=False)
    return dfupdate

def create_training_data(dataset_name):
    clean_df = pd.read_csv(Config.get("datapool.folder") + '/'+dataset_name+'/clean.csv', header=0, dtype=object, na_filter=False)
    print(clean_df.shape)
    dfupdate=clean_df#.sample(3000)
    train = pd.DataFrame (columns = ['tid','cid','column','value','label'])
    
    for i, row in dfupdate.iterrows():
        column_index = random.randint(0,len(dfupdate.columns)-1)
        if i%2==0:
            x = random.randint(0,3)
            if x==0: #shuffle
                dfupdate.at[i,dfupdate.columns[column_index]] = ''.join(random.sample(str(dfupdate.at[i,dfupdate.columns[column_index]]),len(str(dfupdate.at[i,dfupdate.columns[column_index]]))))
            elif x==1: #add a random character at the end
                dfupdate.at[i,dfupdate.columns[column_index]] = str(dfupdate.at[i,dfupdate.columns[column_index]]) + random.choice(string.ascii_letters)
            elif x==2: #remove value
                dfupdate.at[i,dfupdate.columns[column_index]] = ''
            else: #add a random character in the start
                dfupdate.at[i,dfupdate.columns[column_index]] = random.choice(string.ascii_letters) + str(dfupdate.at[i,dfupdate.columns[column_index]])
            train = train.append({'tid': i, 'cid': column_index, 'column': dfupdate.columns[column_index], 'value': dfupdate.at[i,dfupdate.columns[column_index]], 'label': 1}, ignore_index=True)
        else:
            train = train.append({'tid': i, 'cid': column_index, 'column': dfupdate.columns[column_index], 'value': dfupdate.at[i,dfupdate.columns[column_index]], 'label': -1}, ignore_index=True)

    print(dfupdate.shape)
    print(train.shape)    
    file_name=dataset_name+'_train.csv'
    train.to_csv(file_name, index=False)
    return train


In [5]:
from sklearn.model_selection import train_test_split

def load_dataset(force_download=False):
    datasetpath = 'data/Adult/train.csv'
    data = pd.read_csv(datasetpath)
    data['column'] = data['column'].astype(str)
    data['value'] = data['value'].astype(str)
    #data['label'] = data['label'].astype(str)

    print("total.:"+str(data.shape))
    data.loc[data['label'] == -1, 'label'] = 0
    print("pos.:"+str(data[data['label']==1].shape))
    print("neg.:"+str(data[data['label']==0].shape))
    

    X_train, X_test, y_train, y_test = train_test_split(data.loc[:, data.columns != 'label'],
                                                        data["label"], 
                                                        test_size=0.2, 
                                                        random_state=42)
    train_df = pd.concat([X_train, y_train], axis=1, sort=False)
    test_df = pd.concat([X_test, y_test], axis=1, sort=False)
    return train_df, test_df

In [6]:
train, test = load_dataset()

total.:(32560, 5)
pos.:(16280, 5)
neg.:(16280, 5)


In [7]:
train.head()

,tid,cid,column,value,label
24639,24639,1,State-gov,Private,0
5514,5514,0,39,X21,1
19777,19777,0,39,64,0
10781,10781,6,Adm-clerical,Tech-support,0
32239,32239,4,13,11,0


In [8]:
train.columns

Index(['tid', 'cid', 'column', 'value', 'label'], dtype='object')

In [9]:
DATA_COLUMN = 'value'
LABEL_COLUMN = 'label'
label_list = [0, 1]

## Data Preprocessing

We'll need to transform our data into a format transformer understands. This involves two steps. 
First, we create InputExample's:
    - text_a is the text we want to classify, which in this case, is the value field in our Dataframe.
    - text_b is used if we're training a model to understand the relationship between tuples (optional)
    - label is the label for our example, i.e. True, False

In [10]:
train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


To start, we'll need to load a vocabulary file and lowercasing information directly from the tf hub module:


In [11]:
MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    tabReformer_module = hub.Module(MODEL_HUB)
    tokenization_info = tabReformer_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Using our tokenizer, we'll call run_classifier.convert_examples_to_features on our InputExamples to convert them into features TabReformer understands.


In [12]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that tabReformer understands.
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 26048


INFO:tensorflow:Writing example 0 of 26048


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


## Creating a model

Now that we've prepared our data, let's focus on building a model. create_model does just this below. First, it loads the tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our task (i.e. classifying each cell is correct or invalid). This strategy of using a mostly trained model is called fine-tuning.

In [13]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""
    TabReformer_module = hub.Module(
        MODEL_HUB,
        trainable=True)
    TabReformer_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    TabReformer_outputs = TabReformer_module(
        inputs=TabReformer_inputs,
        signature="tokens",
        as_dict=True)
    
    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    
    output_layer = TabReformer_outputs["pooled_output"]
    hidden_size = output_layer.shape[-1].value
    
    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())
    
    with tf.variable_scope("loss"):
        
        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        
        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)



Next we'll wrap our model function in a model_fn_builder function that adapts our model to work for training, evaluation, and prediction.


In [14]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [15]:
# Compute train and warmup steps from batch size
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100


In [16]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [17]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)


In [18]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'err_detect_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f083a2235d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'err_detect_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f083a2235d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (train_features) and produces a generator. This is a pretty standard design pattern for working with Tensorflow Estimators.


In [19]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [20]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
Use standard file APIs to delete files with this prefix.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:Loss for final step: 0.14423275.


INFO:tensorflow:Loss for final step: 0.14423275.


Training took time  10:06:14.248777


Now let's use our test data to see how well our model did:

In [21]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [22]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/home/mona/anaconda3/envs/fastai2/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 2442: auc = 0.96051294, eval_accuracy = 0.96022725, f1_score = 0.95907724, false_negatives = 247.0, false_positives = 12.0, global_step = 2442, loss = 0.10987813, precision = 0.9960617, recall = 0.924741, true_negatives = 3218.0, true_positives = 3035.0


INFO:tensorflow:Saving dict for global step 2442: auc = 0.96051294, eval_accuracy = 0.96022725, f1_score = 0.95907724, false_negatives = 247.0, false_positives = 12.0, global_step = 2442, loss = 0.10987813, precision = 0.9960617, recall = 0.924741, true_negatives = 3218.0, true_positives = 3035.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2442: err_detect_output/model.ckpt-2442


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2442: err_detect_output/model.ckpt-2442


{'auc': 0.96051294,
 'eval_accuracy': 0.96022725,
 'f1_score': 0.95907724,
 'false_negatives': 247.0,
 'false_positives': 12.0,
 'loss': 0.10987813,
 'precision': 0.9960617,
 'recall': 0.924741,
 'true_negatives': 3218.0,
 'true_positives': 3035.0,
 'global_step': 2442}

In [23]:
def getPrediction(cell_values):
    labels = [1, -1]
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in cell_values] 
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(cell_values, predictions)]


In [24]:
pred_sentences = [
  "Self-emp-not-incw",
  "<=50K",
  "0 4",
  "Private"
]


In [25]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from err_detect_output/model.ckpt-2442


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [26]:
predictions

[('Self-emp-not-incw',
  array([-8.198659e+00, -2.750972e-04], dtype=float32),
  -1),
 ('<=50K', array([-3.5649142e-04, -7.9393024e+00], dtype=float32), 1),
 ('0 4', array([-7.7375560e+00, -4.3621086e-04], dtype=float32), -1),
 ('Private', array([-2.8165188e-04, -8.1749430e+00], dtype=float32), 1)]